In [54]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install tqdm
!pip install opencv-contrib-python
!pip install keras-preprocessing
!pip install --upgrade keras
!pip install keras==2.15.0
scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/59/a8/d94e8acb59d678d908fe1db0c7ad89dfa2c2e2e529eeb3c2b3cc218a758d/keras-3.1.1-py3-none-any.whl.metadata
  Using cached keras-3.1.1-py3-none-any.whl.metadata (5.6 kB)
Using cached keras-3.1.1-py3-none-any.whl (1.1 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for keras==2.15.0 from https://files.pythonhosted.org/packages/fc/a7/0d4490de967a67f68a538cc9cdb259bff971c4b5787f7765dc7c8f118f71/keras-2.15.0-py3-none-any.whl.metadata
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1


NameError: name 'scikit' is not defined

In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

In [3]:
TRAIN_DIR = "train"
VAL_DIR="validation"

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"COMPLETED")
    return image_paths,labels

In [5]:
train=pd.DataFrame()
train['image'], train['label']=createdataframe(TRAIN_DIR)

angry COMPLETED
disgust COMPLETED
fear COMPLETED
happy COMPLETED
neutral COMPLETED
sad COMPLETED
surprise COMPLETED


In [6]:
print(train)

                         image     label
0            train\angry\0.jpg     angry
1            train\angry\1.jpg     angry
2           train\angry\10.jpg     angry
3        train\angry\10002.jpg     angry
4        train\angry\10016.jpg     angry
...                        ...       ...
28816  train\surprise\9969.jpg  surprise
28817  train\surprise\9985.jpg  surprise
28818  train\surprise\9990.jpg  surprise
28819  train\surprise\9992.jpg  surprise
28820  train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [7]:
validation=pd.DataFrame()
validation['image'],validation['label']=createdataframe(VAL_DIR)

angry COMPLETED
disgust COMPLETED
fear COMPLETED
happy COMPLETED
neutral COMPLETED
sad COMPLETED
surprise COMPLETED


In [8]:
print(validation)
print(validation['image'])

                             image     label
0       validation\angry\10052.jpg     angry
1       validation\angry\10065.jpg     angry
2       validation\angry\10079.jpg     angry
3       validation\angry\10095.jpg     angry
4       validation\angry\10121.jpg     angry
...                            ...       ...
7061  validation\surprise\9806.jpg  surprise
7062  validation\surprise\9830.jpg  surprise
7063  validation\surprise\9853.jpg  surprise
7064  validation\surprise\9878.jpg  surprise
7065   validation\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         validation\angry\10052.jpg
1         validation\angry\10065.jpg
2         validation\angry\10079.jpg
3         validation\angry\10095.jpg
4         validation\angry\10121.jpg
                    ...             
7061    validation\surprise\9806.jpg
7062    validation\surprise\9830.jpg
7063    validation\surprise\9853.jpg
7064    validation\surprise\9878.jpg
7065     validation\surprise\993.jpg
Name: image, Length: 7066, d

In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features=[]
    for image in tqdm(images):
        img = load_img(image,grayscale=True)
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features),48,48,1)
    return features

In [11]:
train_features=extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\prach\AppData\Roaming\Python\Python311\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [12]:
validation_features=extract_features(validation['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
x_train=train_features/255.0
x_validation=validation_features/255.0

In [14]:
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le=LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [17]:
y_train=le.transform(train['label'])
y_validation=le.transform(validation['label'])

In [18]:
y_train=to_categorical(y_train,num_classes=7)
y_validation=to_categorical(y_validation,num_classes=7)

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D

In [20]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [21]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [22]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save('emotiondetector.keras')


In [23]:
from keras.models import model_from_json

In [24]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [25]:
import keras
print(keras.__version__)


2.15.0


In [43]:
label=['angry','disgust','fear','happy','neutral','sad','suprise']

In [44]:
def ef(image):
    img=load_img(image,grayscale=True)
    feature=np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [45]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image='train/fear/10010.jpg'
print("Image of angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("This EmotiTrack predicts",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')